## Cross-cancer prediction analysis

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import seaborn as sns

import pancancer_evaluation.config as cfg
import pancancer_evaluation.utilities.analysis_utilities as au

In [6]:
cross_cancer_dir = os.path.join(cfg.results_dir, 'cross_cancer', 'cross_cancer')
cross_cancer_df = au.load_prediction_results_cc(cross_cancer_dir, 'cross_cancer')
print(cross_cancer_df.shape)
cross_cancer_df.head()

(3456, 9)


,auroc,aupr,train_identifier,test_identifier,signal,seed,data_type,experiment,train_test_equal
0,0.92601,0.555590,NRAS_COAD,PTEN_SKCM,shuffled,42,train,cross_cancer,False
1,0.50530,0.092270,NRAS_COAD,PTEN_SKCM,shuffled,42,test,cross_cancer,False
2,0.48782,0.061438,NRAS_COAD,PTEN_SKCM,shuffled,42,cv,cross_cancer,False
0,0.53141,0.063640,NF1_LGG,TTN_GBM,shuffled,42,train,cross_cancer,False
1,0.54635,0.332380,NF1_LGG,TTN_GBM,shuffled,42,test,cross_cancer,False


In [11]:
# get rows that have the same identifier
cross_cancer_df['train_test_equal'] = (cross_cancer_df.train_identifier ==
                                       cross_cancer_df.test_identifier)
# get rows that have the same gene (possibly different identifiers)
cross_cancer_df['gene_equal'] = (cross_cancer_df.train_identifier.str.split('_', expand=True)[0] ==
                                 cross_cancer_df.test_identifier.str.split('_', expand=True)[0])

,auroc,aupr,train_identifier,test_identifier,signal,seed,data_type,experiment,train_test_equal,gene_equal
0,0.99447,0.918370,ATM_COAD,ATM_SKCM,signal,42,train,cross_cancer,False,True
1,0.41006,0.046425,ATM_COAD,ATM_SKCM,signal,42,test,cross_cancer,False,True
2,0.52773,0.097605,ATM_COAD,ATM_SKCM,signal,42,cv,cross_cancer,False,True
0,0.99927,0.986630,PTEN_COAD,BRAF_THCA,signal,42,train,cross_cancer,False,False
1,0.75510,0.742380,PTEN_COAD,BRAF_THCA,signal,42,test,cross_cancer,False,False
2,0.69255,0.170960,PTEN_COAD,BRAF_THCA,signal,42,cv,cross_cancer,False,False
0,0.94043,0.674030,ATM_SKCM,TP53_LGG,shuffled,42,train,cross_cancer,False,False
1,0.48997,0.496980,ATM_SKCM,TP53_LGG,shuffled,42,test,cross_cancer,False,False
2,0.43002,0.058112,ATM_SKCM,TP53_LGG,shuffled,42,cv,cross_cancer,False,False
0,0.99941,0.998930,TP53_GBM,NRAS_THCA,signal,42,train,cross_cancer,False,False
